In [1]:
import torch as tc
import numpy as np
import pandas as pd

from star.star import STAR
from curve_utils import CurveUtils
from curve_generator import CurveGenerator, get_all_curves, select_better
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

# Prepating Bodies

In [2]:
for gender in ['female']:
    betas = tc.FloatTensor(np.load(f'data/{gender}_betas.npy')).to(device)
    trans = tc.FloatTensor(np.load(f'data/{gender}_trans.npy')).to(device)
    star = STAR(gender=gender, passed_path=f"star/{gender}.npz", num_betas=betas.shape[-1])
    
    poses = tc.FloatTensor(np.load(f'data/{gender}_poses.npy')).to(device)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_a.pt')
    
    poses = np.zeros((betas.shape[0],72))
    poses[:,5] = 6 * np.pi/180
    poses[:,8] = -6 * np.pi/180
    poses = tc.cuda.FloatTensor(poses)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_t.pt')

template = star.faces.to(device)

In [3]:
%%script false --no-raise-error
for gender in ['female']:
    bodies = tc.load(f'data/{gender}_bodies_a.pt')
    for idx, body in enumerate(bodies):
            CurveUtils.save_obj(f"tests/{gender}_{idx}_body.obj", body, star.faces+1)

# Loading Measures

In [4]:
their_semantics = [
    'Neck girth',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Upper arm girth R',
    'Thigh girth R',
    'Height (m)',
    'Inside leg height',
    'Across back shoulder width',
]
our_semantic = [
    'neck_girth', # 5.3.2
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'upper_arm_girth', # 5.3.16
    'thigh_girth', # 5.3.20
    'height', # 5.1.1
    'crotch_height', # 5.1.15
    'shoulder_width', # 5.4.3
]

In [5]:
selected_subjects = dict()
selected_measures = dict()

for gender in ['female']:
    measures = pd.read_pickle(f'data/{gender}_measures.pkl')
    measures = measures[measures['Measuring station'] == "Expert measurer 5"]
    temp_measures = measures[their_semantics].iloc[::2]
    
    mean_measures = measures.mean(axis=0)
    # selected_subjects[gender] = abs(temp_measures - mean_measures).sum(axis=1).idxmin()
    selected_subjects[gender] = 'IEEEP2_01'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['height'] *= 1000

/tmp/ipykernel_906059/4115461512.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_measures = measures.mean(axis=0)


In [6]:

results = get_all_curves(selected_subjects, selected_measures, template, device)
for gender in ['female']:
    tc.save(results[gender], f'./data/{gender}_result.zip')

PROCESSING FEMALE


In [7]:
select_better(selected_subjects, selected_measures, device)


FEMALE
                  original    measured  error(mm)
neck_girth            33.7   33.767044   0.670441
bust_chest_girth      88.1    88.13529   0.352921
waist_girth           74.1   74.084854   0.151459
hip_girth            104.2    104.2078   0.078018
upper_arm_girth       29.3    29.25493   0.450695
thigh_girth           62.2     62.1598   0.401985
height               164.0  165.284061  12.840614


# Validation

In [8]:
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
}

In [9]:
all_measures = dict()

for gender in ['female']:
    measures = pd.read_pickle(f'data/{gender}_measures.pkl')
    # measures = measures[measures['Sex']=='female']
    all_measures[gender] = dict()
    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        temp_measures = measures[measures['Measuring station'] == station]
        temp_measures = temp_measures[their_semantics].iloc[::2]
        temp_measures = temp_measures[their_semantics]
        temp_measures.columns = our_semantic
        all_measures[gender][station] = temp_measures

In [10]:
measures

,Measuring station,repetition no.,Neck girth,Back neck point to waist,Upper arm girth R,Back neck point to wrist R,Across back shoulder width,Bust girth,Waist girth,Hip girth,Thigh girth R,Total crotch length,Inside leg height,Lowaist,Sex,Age (years),Weight (kg),Height (m)
Subject,,,,,,,,,,,,,,,,,,
IEEEP2_01,MOVE4D,1,327.377,425.868,285.042,726.648,387.684,891.216,755.405,1057.48,629.809,771.791,714.434,833.234,female,23,64.7,1.64
IEEEP2_01,MOVE4D,2,326.233,435.319,288.409,726.637,385.951,891.291,765.505,1060.04,624.778,764.475,711.900,840.060,female,23,64.7,1.64
IEEEP2_01,Expert measurer 1,1,330.000,407.500,275.000,750.000,435.000,880.000,730.000,1080.00,620.000,750.000,720.000,845.000,female,23,64.7,1.64
IEEEP2_01,Expert measurer 1,2,335.000,400.000,270.000,755.000,430.000,880.000,745.000,1080.00,625.000,760.000,700.000,820.000,female,23,64.7,1.64
IEEEP2_01,Expert measurer 5,1,337.000,395.000,293.000,740.000,425.000,881.000,741.000,1042.00,622.000,740.000,740.000,844.000,female,23,64.7,1.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IEEEP2_71,MOVE4D,2,400.280,423.083,404.993,718.047,397.608,1208.460,1132.850,1367.27,736.902,869.268,684.827,1297.330,female,55,106.1,1.61
IEEEP2_71,Expert measurer 5,1,375.000,376.000,429.000,728.000,402.000,1210.000,1135.000,1352.00,747.000,925.000,700.000,1200.000,female,55,106.1,1.61
IEEEP2_71,Expert measurer 5,2,360.000,365.000,430.000,734.000,403.000,1212.000,1128.000,1351.00,750.000,942.000,695.000,1198.000,female,55,106.1,1.61


In [11]:
tc.set_printoptions(precision=2, sci_mode=False, linewidth=100)

for gender in ['female']:
    bodies = tc.load(f'data/{gender}_bodies_a.pt')
    best_curves = tc.load(f'data/{gender}_best_curves.zip')

    all_bodies_measures = []
    for idx,body in enumerate(bodies):
        all_curves_measures = []
        all_positions = []
        for curve in best_curves:
            position = CurveGenerator.generate_positions(body, curve)[::2]
            all_positions.extend(position.cpu().numpy())
            measures = CurveGenerator.calculate_distances(position)
            all_curves_measures.append(measures)
        all_bodies_measures.append(all_curves_measures)
        CurveUtils.save_obj(f"tests/{gender}_{idx}_curves.obj", all_positions)
    all_bodies_measures = tc.FloatTensor(all_bodies_measures)
    our_measures = all_bodies_measures

    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        print(f"\n\nmeasures difference from {station}:")
        inkreate_measures = tc.FloatTensor(all_measures[gender][station][list(curve_index.keys())].to_numpy())
        
        diff = abs(inkreate_measures[::2] - (our_measures*10))

        print(pd.DataFrame({
            "mean": diff.mean(axis=0),
            "std": diff.var(axis=0),
            "min": diff.min(axis=0).values,
            "max": diff.max(axis=0).values,
        }, index=list(curve_index.keys())))



measures difference from MOVE4D:
                       mean         std        min        max
neck_girth        22.091906   46.599888  11.554688  37.332367
bust_chest_girth  16.016066  107.960136   5.034668  49.583435
waist_girth       31.879921  320.347260   2.086060  61.192017
hip_girth         15.253754  171.941711   0.583374  49.922974
upper_arm_girth    3.651355    5.985104   0.289093   9.954559
thigh_girth       16.447554   41.250980   2.481323  26.888855


measures difference from Expert measurer 1:
                       mean         std       min        max
neck_girth        23.717958  122.851166  8.702759  51.670715
bust_chest_girth  15.305034  155.295929  0.783203  45.033813
waist_girth       13.165168  103.361374  0.792542  39.305176
hip_girth         10.881415  121.598579  0.947754  40.372925
upper_arm_girth   13.188964   52.864838  3.804047  29.384125
thigh_girth       17.989359   63.415092  3.561157  29.769409


measures difference from Expert measurer 5:
            